In [2]:
import pandas as pd


In [4]:
import pandas as pd

df = pd.read_csv("blood_pressure_data.csv")
df.head()


,Date,Time,Patient First Name,Patient Last Name,PCP,Clinic Visit Provider,Clinic,Systolic BP reading,Diastolc BP reading
0,6/23/2021,11:41:00 AM,A,A,Dr. Clark,Dr. Clark,GIM,149,89
1,10/26/2021,1:45:00 PM,A,A,Dr. Clark,Dr. Basit,Parkland Cardiology,141,64
2,9/11/2021,1:05:00 PM,C,A,Dr. Clark,Dr. Basit,UTSW Cardiology,110,67
3,9/11/2021,1:07:00 PM,C,A,Dr. Clark,Dr. Basit,UTSW Cardiology,110,63
4,1/15/2021,10:33:00 AM,G,A,Dr. Fish,Dr. Basit,Parkland Cardiology,168,103


In [5]:
df.columns


Index(['Date', 'Time', 'Patient First Name', 'Patient Last Name', 'PCP',
       'Clinic Visit Provider', 'Clinic', 'Systolic BP reading',
       'Diastolc BP reading'],
      dtype='str')

In [6]:
df.shape


(154, 9)

In [7]:
df.isna().sum()


Date                     0
Time                     0
Patient First Name       0
Patient Last Name        0
PCP                      0
Clinic Visit Provider    0
Clinic                   0
Systolic BP reading      0
Diastolc BP reading      0
dtype: int64